In [2]:
import pandas as pd

users = pd.read_csv('../preproc_users_view.csv')
tweets_view = pd.read_csv('../preproc_tweets_view.csv')

users_view = users[users.user_id.isin(tweets_view.user_id.unique())].copy()

users_view['hashtags'] = users_view['hashtags'].map(eval)
users_view['original_hashtags'] = users_view['original_hashtags'].map(eval)
users_view['retweet_hashtags'] = users_view['retweet_hashtags'].map(eval)
users_view['quote_hashtags'] = users_view['quote_hashtags'].map(eval)

tweets_view['hashtags'] = tweets_view['hashtags'].map(eval)
tweets_view['original_hashtags'] = tweets_view['original_hashtags'].map(eval)
tweets_view['retweet_hashtags'] = tweets_view['retweet_hashtags'].map(eval)
tweets_view['quote_hashtags'] = tweets_view['quote_hashtags'].map(eval)

# users_view['user_mentions'] = users_view['user_mentions'].map(eval)
# users_view['original_user_mentions'] = users_view['original_user_mentions'].map(eval)
# users_view['retweet_user_mentions'] = users_view['retweet_user_mentions'].map(eval)
# users_view['quote_user_mentions'] = users_view['quote_user_mentions'].map(eval)

# tweets_view['user_mentions'] = tweets_view['user_mentions'].map(eval)
# tweets_view['original_user_mentions'] = tweets_view['original_user_mentions'].map(eval)
# tweets_view['retweet_user_mentions'] = tweets_view['retweet_user_mentions'].map(eval)
# tweets_view['quote_user_mentions'] = tweets_view['quote_user_mentions'].map(eval)

stats_df = pd.DataFrame(
    [
        '{:,}'.format(len(users[users.is_croatian == True])),
        '{:,}'.format(len(users_view)),
        '{:,}'.format(len(tweets_view)),
        
        '{0:.2f}%'.format(100*(tweets_view['is_original'].sum() / (tweets_view['is_original'].sum() + tweets_view['is_retweet'].sum()))),
        '{0:.2f}%'.format(100*(tweets_view['is_retweet'].sum() / (tweets_view['is_original'].sum() + tweets_view['is_retweet'].sum()))),
        '{:,}'.format(tweets_view['is_reply'].sum()),
        '{:,}'.format(tweets_view['is_quote'].sum()),
        
        tweets_view.created_at.min(),
        tweets_view.created_at.max()

    ],
    index=[
        "Croatian Users (global) #",
        "Croatian Users (tweets) #",
        "Tweets #",
        
        "Original %",
        "Reweets %",
        "Replies #",
        "Quotes #",
        
        "Earliest Tweet date",
        "Latest Tweet date"
    ],
    columns=['Tweets Data']
)

stats_df

,Tweets Data
Croatian Users (global) #,"32,328"
Croatian Users (tweets) #,"7,720"
Tweets #,"833,564"
Original %,68.92%
Reweets %,31.08%
Replies #,"353,261"
Quotes #,"53,907"
Earliest Tweet date,2022-10-01 00:00:19+00:00
Latest Tweet date,2022-12-04 00:59:41+00:00


In [3]:
users_view.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7720 entries, 4 to 32359
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   user_id                   7720 non-null   int64  
 1   screen_name               7720 non-null   object 
 2   name                      7720 non-null   object 
 3   verified                  7720 non-null   bool   
 4   location                  7720 non-null   object 
 5   clean_location            7720 non-null   object 
 6   description               6340 non-null   object 
 7   is_croatian               7720 non-null   bool   
 8   followers_count           7720 non-null   int64  
 9   friends_count             7720 non-null   int64  
 10  favourites_count          7720 non-null   int64  
 11  total_out_tweets_cnt      7720 non-null   float64
 12  out_retweets_cnt          7720 non-null   float64
 13  out_retweets_pct          7720 non-null   float64
 14  out_rep

In [ ]:
user_nodes = users_view['user_id', 'screen_name', '']

In [ ]:
tweets_view.info()

In [4]:
hashtags_original_view = tweets_view[tweets_view['is_original'] == True].groupby(['year', 'month', 'week', 'day', 'screen_name']).agg(
    original_favorite_cnt=('original_favorite_cnt', 'sum'),
    original_hashtags=('original_hashtags', 'sum')
)

hashtags_retweet_view = tweets_view[tweets_view['is_retweet'] == True].groupby(['year', 'month', 'week', 'day', 'screen_name', 'retweet_from_screen_name']).agg(
    retweet_favorite_cnt=('retweet_favorite_cnt', 'sum'),
    retweet_timedelta_sec=('retweet_timedelta_sec', 'mean'),
    retweet_hashtags=('retweet_hashtags', 'sum')
)

hashtags_quote_view = tweets_view[tweets_view['is_quote'] == True].groupby(['year', 'month', 'week', 'day', 'screen_name', 'quote_from_screen_name']).agg(
    quote_favorite_cnt=('quote_favorite_cnt', 'sum'),
    quote_timedelta_sec=('quote_timedelta_sec', 'mean'),
    quote_hashtags=('quote_hashtags', 'sum'),
)


In [5]:
original__user_hashtag = hashtags_original_view.reset_index().explode('original_hashtags') \
    .dropna(subset='original_hashtags') \
    .rename(columns={
        'screen_name': 'source',
        'original_hashtags': 'target'
    }).reset_index().groupby(['year', 'month', 'week', 'day', 'source', 'target']).agg(
        tweet_count=('index', 'size'),
        favorite_count=('original_favorite_cnt', 'sum')
    ).reset_index()
original__user_hashtag = original__user_hashtag[original__user_hashtag.source.isin(original__user_hashtag.target)]
original__user_hashtag.to_csv('original__user_hashtag.csv', index=False)
original__user_hashtag

,year,month,week,day,source,target,tweet_count,favorite_count
337,2022,10,39,1,LikeZadar,FeelZadar,1,33
338,2022,10,39,1,LikeZadar,LikeZadar,1,33
512,2022,10,39,1,RitamBalkana,AcaLukas,1,0
513,2022,10,39,1,RitamBalkana,RitamBalkana,1,0
514,2022,10,39,1,RitamBalkana,RitamBalkanaCom,1,0
...,...,...,...,...,...,...,...,...
78875,2022,12,48,3,ravnododna,seine,1,1
78970,2022,12,48,3,znatko,odgovor,3,0
78971,2022,12,48,3,znatko,pitanje,3,0
78978,2022,12,48,4,znatko,odgovor,1,0


In [6]:
retweet__user_user = hashtags_retweet_view.reset_index().explode('retweet_hashtags') \
    .dropna(subset='retweet_hashtags') \
    .rename(columns={
        'retweet_from_screen_name': 'source',
        'screen_name': 'target',
        'retweet_hashtags': 'hashtag'
    }).reset_index().groupby(['year', 'month', 'week', 'day', 'source', 'target', 'hashtag']).agg(
        retweet_count=('index', 'size'),
        favorite_count=('retweet_favorite_cnt', 'sum')
    ).reset_index()
retweet__user_user = retweet__user_user[retweet__user_user.source.isin(retweet__user_user.target)]
retweet__user_user.to_csv('retweet__user_user.csv', index=False)
retweet__user_user

,year,month,week,day,source,target,hashtag,retweet_count,favorite_count
61,2022,10,39,1,AmineKhelif,Ambalgzagreb,Algeria,1,26.0
62,2022,10,39,1,AmineKhelif,Ambalgzagreb,Hravtska,1,26.0
63,2022,10,39,1,AmineKhelif,Ambalgzagreb,Opatija,1,26.0
64,2022,10,39,1,AmineKhelif,Marina87251891,Algeria,2,98.0
65,2022,10,39,1,AmineKhelif,Marina87251891,Hravtska,1,49.0
...,...,...,...,...,...,...,...,...,...
133018,2022,12,48,3,viborc,ZoranPehar,Hrvatska,1,6.0
133109,2022,12,48,4,HNS_CFF,PrvaHrvLiga,FIFAWorldCup,1,25.0
133110,2022,12,48,4,HNS_CFF,PrvaHrvLiga,Obitelj,1,25.0
133111,2022,12,48,4,HNS_CFF,PrvaHrvLiga,Qatar2022,1,25.0


In [20]:
retweet__user_user[retweet__user_user['source'] == 'hrzz_science']

,year,month,week,day,source,target,hashtag,retweet_count,favorite_count
13293,2022,6,23,8,hrzz_science,Poljuha_Lab,hrzz,1,8.0
47141,2022,6,26,30,hrzz_science,GlasHrvatske,hrzz,1,10.0
47142,2022,6,26,30,hrzz_science,pintofscienceHR,hrzz,1,10.0
48789,2022,7,26,1,hrzz_science,daavork,hrzz,1,3.0
73566,2022,7,29,18,hrzz_science,pintofscienceHR,hrzz,1,6.0
79073,2022,7,29,22,hrzz_science,ivana_brekalo,hrzz,1,3.0
80285,2022,7,29,23,hrzz_science,pintofscienceHR,hrzz,1,3.0
81625,2022,7,29,24,hrzz_science,pintofscienceHR,hrzz,1,13.0
83054,2022,7,30,25,hrzz_science,Mateja_Pisacic,hrzz,1,10.0
83055,2022,7,30,25,hrzz_science,daavork,hrzz,1,10.0


In [7]:
retweet__user_hashtag = hashtags_retweet_view.reset_index().explode('retweet_hashtags') \
    .dropna(subset='retweet_hashtags') \
    .rename(columns={
        'screen_name': 'source',
        'retweet_hashtags': 'target'
    }).reset_index().groupby(['year', 'month', 'week', 'day', 'source', 'target']).agg(
        retweet_count=('index', 'size'),
        favorite_count=('retweet_favorite_cnt', 'sum')
    ).reset_index()
retweet__user_hashtag = retweet__user_hashtag[retweet__user_hashtag.source.isin(retweet__user_hashtag.target)]
retweet__user_hashtag.to_csv('retweet__user_hashtag.csv', index=False)
retweet__user_hashtag

,year,month,week,day,source,target,retweet_count,favorite_count
215,2022,10,39,1,ElonTravel,StandWithUkraine,2,31448.0
782,2022,10,39,1,RebootDevelop,GameDev,1,2.0
783,2022,10,39,1,RebootDevelop,IndieDev,1,2.0
784,2022,10,39,1,RebootDevelop,IndieGameDev,1,7.0
785,2022,10,39,1,RebootDevelop,Inscryption,1,12.0
...,...,...,...,...,...,...,...,...
115580,2022,12,48,3,VladaRH,Eurozone,1,207.0
115581,2022,12,48,3,VladaRH,HRZ,1,34.0
115582,2022,12,48,3,VladaRH,HV,1,34.0
115583,2022,12,48,3,VladaRH,NPOO,1,207.0


In [8]:
quote__user_user = hashtags_quote_view.reset_index().explode('quote_hashtags') \
    .dropna(subset='quote_hashtags') \
    .rename(columns={
        'quote_from_screen_name': 'source',
        'screen_name': 'target',
        'quote_hashtags': 'hashtag'
    }).reset_index().groupby(['year', 'month', 'week', 'day', 'source', 'target', 'hashtag']).agg(
        quote_count=('index', 'size'),
        favorite_count=('quote_favorite_cnt', 'sum')
    ).reset_index()
quote__user_user = quote__user_user[quote__user_user.source.isin(quote__user_user.target)]
quote__user_user.to_csv('quote__user_user.csv', index=False)
quote__user_user

,year,month,week,day,source,target,hashtag,quote_count,favorite_count
3,2022,10,39,1,Andrea_C_White,Andrea_C_White,TheRoadITook,1,15.0
4,2022,10,39,1,Andrea_C_White,Andrea_C_White,artistsontwitter,1,15.0
5,2022,10,39,1,Andrea_C_White,Andrea_C_White,philipquast,1,15.0
6,2022,10,39,1,Andrea_C_White,Andrea_C_White,portrait,1,15.0
19,2022,10,39,1,EPavlic,EPavlic,AIDay,1,61.0
...,...,...,...,...,...,...,...,...,...
14119,2022,12,48,3,smedj,StateOfCroatia,futbol,1,55.0
14123,2022,12,48,3,socijali_ST,ChefJeromeee,politikahr,1,6.0
14126,2022,12,48,3,spas031,spas031,ManUtd,1,0.0
14128,2022,12,48,3,tomislav_stojak,tomyvuk99,Advent,1,3.0


In [9]:
quote__user_hashtag = hashtags_quote_view.reset_index().explode('quote_hashtags') \
    .dropna(subset='quote_hashtags') \
    .rename(columns={
        'screen_name': 'source',
        'quote_hashtags': 'target'
    }).reset_index().groupby(['year', 'month', 'week', 'day', 'source', 'target']).agg(
        quote_count=('index', 'size'),
        favorite_count=('quote_favorite_cnt', 'sum')
    ).reset_index()
quote__user_hashtag = quote__user_hashtag[quote__user_hashtag.source.isin(quote__user_hashtag.target)]
quote__user_hashtag.to_csv('quote__user_hashtag.csv', index=False)
quote__user_hashtag

,year,month,week,day,source,target,quote_count,favorite_count
912,2022,10,40,5,SuperSportHNL,UCL,2,524.0
913,2022,10,40,5,SuperSportHNL,dinamozagreb,2,524.0
2122,2022,10,41,11,SuperSportHNL,UCL,2,420.0
2123,2022,10,41,11,SuperSportHNL,dinamozagreb,2,420.0
3909,2022,10,42,19,InfobipShift,Chrome,1,1.0
3910,2022,10,42,19,InfobipShift,GoogleAurora,1,1.0
3911,2022,10,42,19,InfobipShift,InfobipShift,1,1.0
5181,2022,10,43,25,SuperSportHNL,UCL,2,360.0
5182,2022,10,43,25,SuperSportHNL,dinamozagreb,2,360.0
5689,2022,10,43,27,SuperSportHNL,budiponosan,1,5.0


In [ ]:
import networkx as nx

G = nx.from_pandas_edgelist(
    edgelist.sample(2000), 
    edge_attr=['hashtag', 'retweet_count', 'favorite_count'], 
    create_using=nx.DiGraph
)

pos_spring = nx.spring_layout(G)
fig, ax = plt.subplots(1, figsize=(25, 25))
nx.draw(G, pos=pos_spring, node_size=15, ax=ax)

In [ ]:
degree_sequence = sorted((deg[1] for deg in G.degree()), reverse=True)

def get_average_degree(nx_graph):
    return sum(deg[1] for deg in nx_graph.degree()) / len(nx_graph.degree())


def get_clustering_coeff(nx_graph):
    largest_hubs, _ = get_largest_hubs(nx_graph)
    largest_hub = largest_hubs[0][0]
    return nx.clustering(nx_graph, largest_hub)


def get_largest_hubs(nx_graph, num=10):
    min_largest_degree = degree_sequence[num]
    return sorted(
        filter(lambda x: x[1] > min_largest_degree, nx_graph.degree()), 
        key=lambda x: x[1], 
        reverse=True
    )


def get_measures_df(nx_graph):
    data = [
        nx_graph.number_of_nodes(),
        nx_graph.number_of_edges(),
        round(get_average_degree(nx_graph), 2),
        round(nx.density(nx_graph), 2),
        #nx.number_connected_components(nx_graph)
    ]

    index = [
        'Broj čvorova (N)',
        'Broj veza (K)',
        'Prosječni stupanj ()',
        'Gustoća mreže (d)',
        # 'Broj komponenti'
    ]
    
    try:
        clustering_coeff = round(get_clustering_coeff(nx_graph), 2)
        assortativity_coeff = round(nx.degree_assortativity_coefficient(nx_graph), 2)
    except Exception:
        pass
    else:
        data.insert(-1, clustering_coeff)
        index.insert(-1, 'Koeficijent grupiranja (c)')
        data.insert(-1, assortativity_coeff)
        index.insert(-1, 'Koeficijent asortativnosti (a)')

    return pd.DataFrame(data=data, index=index, columns=['Iznos'])

get_measures_df(G)

In [ ]:
largest_hubs = get_largest_hubs(G, 10)
print("HUBs:", largest_hubs)
largest_hubs = [hub[0] for hub in largest_hubs]

In [ ]:
fig, ax = plt.subplots(1, figsize=(30, 30))
nx.draw_networkx(
    G, 
    nodelist=[node for edge in G.edges(largest_hubs) for node in edge], 
    edgelist=G.edges(largest_hubs), 
    ax=ax
)

In [ ]:
from networkx.algorithms.community import girvan_newman
import itertools
import random

def most_central_edge(G):
    centrality = nx.edge_betweenness_centrality(G, weight='weight')
    return max(centrality, key=centrality.get)

def get_communities(nx_graph, k=4):
    comp = girvan_newman(nx_graph, most_valuable_edge=most_central_edge)
    all_communities = []
    for communities in itertools.islice(comp, k):
        community = tuple(sorted(c) for c in communities)
        all_communities.append(community)
    
    # gn_communities = {
    #     node: i for i, nodes in enumerate(gn_communities) for node in nx_graph.nodes if i < 10
    # }
    # gn_community_colors = {
    #     community: '#' + ''.join([random.choice('0123456789ABCDEF') for _ in range(6)])
    #     for community in gn_communities.values()
    # }
    # gn_community_colors.update({-1: '#000000'})

    return all_communities

all_communities = get_communities(G)
print(all_communities)